In [ ]:
import sys

sys.path.insert(0, "..")
from runner.utils import (
    allocate_benchmarks,
    create_benchmark_campaign,
    display_speedups,
    load_benchmark_metadata,
    load_results,
)

In [2]:
# If a util function was modified, use this cell to reload it without having to restart the kernel
%run ../runner/utils.py

## Create benchmark campaign(s)

In [3]:
benchmarks_df = load_benchmark_metadata()
benchs_to_run = benchmarks_df[
    (benchmarks_df["Benchmark"] == "pypsa-de-elec")
    & (benchmarks_df["Instance"].str.endswith("-1h"))
]
# NOTE: picking only even num of nodes >= 6 to save costs
benchs_to_run = benchs_to_run[
    benchs_to_run["Instance"].map(
        lambda i: (lambda n: n >= 6 and n % 2 == 0)(int(i.split("-")[0]))
    )
]
len(benchs_to_run)

11

In [4]:
# Create campaign: 1 instance per VM, latest solvers only (because I marked all as Ls in the metadata)

benchs_to_run.loc[:, "Num. variables"] = 1  # Dummy value to run 1 instance per VM
vm_yamls = allocate_benchmarks(
    benchs_to_run,
    "Num. variables",
    len(benchs_to_run),
    machine_type="c4-standard-8",  # NOTE: picked a smaller machine size to save costs!
    timeout_seconds=24 * 60 * 60,
    solvers="cbc gurobi highs-hipo highs-ipm highs",  # NOTE: skipped SCIP since it's not built for LPs
    years=[2024, 2025],  # latest solvers only, so skip creating older conda envs
)

# # Use a different zone for some instances to avoid exceeding CPU quota
# for y in vm_yamls:
#     if int(y['benchmarks']['pypsa-de-elec']['Sizes'][0]['Name'].split('-')[0]) >= 20:
#         print(y['benchmarks']['pypsa-de-elec']['Sizes'][0]['Name'])
#         y['zone'] = 'us-east4-a'

create_benchmark_campaign(
    "20260211-all-pypsa-de-sizes",
    "all-pypsa-de-sizes",
    vm_yamls,
)

Allocated. Estimated runtime: 0.0h
  VM 00: 1 instances, 0.0h
  VM 01: 1 instances, 0.0h
  VM 02: 1 instances, 0.0h
  VM 03: 1 instances, 0.0h
  VM 04: 1 instances, 0.0h
  VM 05: 1 instances, 0.0h
  VM 06: 1 instances, 0.0h
  VM 07: 1 instances, 0.0h
  VM 08: 1 instances, 0.0h
  VM 09: 1 instances, 0.0h
  VM 10: 1 instances, 0.0h
Created directory and files in ../infrastructure/benchmarks/20260211-all-pypsa-de-sizes
Run this campaign from the infrastructure/ directory using the command:
tofu apply -var-file benchmarks/20260211-all-pypsa-de-sizes/run.tfvars -state=states/20260211-all-pypsa-de-sizes.tfstate


NOTE: to run tofu using service key, I used the env var method: `export GOOGLE_APPLICATION_CREDENTIALS=$(realpath ../orchestrator-gcp-key.json)`

## Monitor runs

To view running VMs:
```
gcloud compute instances list | sort | tee /dev/tty | grep benchmark-instance | grep -i running | wc -l
```

## Inspect results

To download results:
```
gsutil -m rsync -r gs://solver-benchmarks/results ./results/gcp-results/
```

In [5]:
results, variability = load_results(["../results/gcp-results/20251219-pypsa-sizes//"])
display_speedups(results, benchmarks_df)

Found 33 records, 11 benchmark instances


ValueError: Missing Num. variables for:
pypsa-de-elec-10-12h-10-12h
pypsa-de-elec-10-1h-10-1h
pypsa-de-elec-10-3h-10-3h
pypsa-de-elec-20-12h-20-12h
pypsa-de-elec-20-1h-20-1h
pypsa-de-elec-20-24h-20-24h
pypsa-de-elec-20-3h-20-3h
pypsa-de-elec-50-12h-50-12h
pypsa-de-elec-50-1h-50-1h
pypsa-de-elec-50-24h-50-24h
pypsa-de-elec-50-3h-50-3h